# Music recommender system

## Content-based filters

In [24]:
import pandas as pd
import numpy as np
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')


In [25]:
songs = pd.read_csv("song_lyric_data.csv")

In [26]:
songs.shape

(57650, 4)

In [27]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


## Because of the dataset being so big, we are going to resample only 5000 random songs.

In [28]:
songs = songs.sample(n=5000).drop('link' , axis=1).reset_index(drop=True)

## We can notice also the presence of \n in the text, so we are going to remove it.

In [29]:
songs['text'] = songs['text'].str.replace(r'\n', '')

## After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word.

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
tfid = TfidfVectorizer(analyzer='word' , stop_words='english')

In [32]:
lyrics_matrix = tfid.fit_transform(songs['text'])

## We now need to calculate the similarity of one lyric to another. We are going to use cosine similarity.

In [33]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [34]:
cosine_similarities.shape

(5000, 5000)

## Once we get the similarities, we'll store in a dictionary the names of the 50 most similar songs for each song in our dataset.

In [35]:
similarities = {}

In [36]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]


In [37]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [38]:
recommedations = ContentBasedRecommender(similarities)

## Then, we are ready to pick a song from the dataset and make a recommendation.

In [39]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 10 
}

In [40]:
recommedations.recommend(recommendation)

The 10 recommended songs for I Love You, Suzanne are:
Number 1:
Suzanne by Journey with 0.765 similarity score
--------------------
Number 2:
Poppin' by Utada Hikaru with 0.303 similarity score
--------------------
Number 3:
Hey You by Modern Talking with 0.296 similarity score
--------------------
Number 4:
The Best Damn Thing by Avril Lavigne with 0.296 similarity score
--------------------
Number 5:
Kansas City by Paul McCartney with 0.273 similarity score
--------------------
Number 6:
Hey Now by Ray Charles with 0.258 similarity score
--------------------
Number 7:
Do You Want To by Xscape with 0.253 similarity score
--------------------
Number 8:
Loaded by Wishbone Ash with 0.232 similarity score
--------------------
Number 9:
Only Love by John Prine with 0.214 similarity score
--------------------
Number 10:
Why Can't This Be Love by Van Halen with 0.214 similarity score
--------------------
